In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(temperature=0.1)

examples = [
    {"movie": "탑건", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "emojis": "👨‍👨‍👦🔫🍝"},
    {"movie": "해리포터", "emojis": "🧙‍♂️🪄🏰"},
    {"movie": "타이타닉", "emojis": "🚢💔🌊"},
    {"movie": "매트릭스", "emojis": "💊🕶️💻"},
    {"movie": "스타워즈", "emojis": "🌌🚀🧔"},
    {"movie": "어벤져스", "emojis": "🦸‍♂️🦸‍♀️🌍"},
    {"movie": "인셉션", "emojis": "🌀🛌🕰️"},
    {"movie": "쥬라기 공원", "emojis": "🦖🌴🚁"},
    {"movie": "라라랜드", "emojis": "🎹💃🌆"}
]

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=50,
    memory_key="history",
    return_messages=True,
)

example_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a machine that takes the name of the movie and makes three emojis that can represent the movie"),
    # MessagesPlaceholder(variable_name="history"),
    ("human", "{movie}"),
    ("ai", "{emojis}")
])
print(example_prompt)



input_variables=['emojis', 'movie'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a machine that takes the name of the movie and makes three emojis that can represent the movie')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['movie'], template='{movie}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['emojis'], template='{emojis}'))]


```python
import typing
from langchain.schema.messages import (
    AIMessage,
    HumanMessage,
    ChatMessage,
    SystemMessage,
    FunctionMessage
)

input_variables = ["emojis", "history", "movie"]

input_types = {
    "history": typing.List[
        typing.Union[
            AIMessage,
            HumanMessage,
            ChatMessage,
            SystemMessage,
            FunctionMessage
        ]
    ]
}

messages = [
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],
            template="You are a machine that takes the name of the movie and makes three emojis that can represent the movie"
        )
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["movie"],
            template="{movie}"
        )
    ),
    AIMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["emojis"],
            template="{emojis}"
        )
    )
]

```


In [51]:
fewshot_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    input_variables=["movie"]
)

print(fewshot_template)

examples=[{'movie': '탑건', 'emojis': '🛩️👨\u200d✈️🔥'}, {'movie': '대부', 'emojis': '👨\u200d👨\u200d👦🔫🍝'}, {'movie': '해리포터', 'emojis': '🧙\u200d♂️🪄🏰'}, {'movie': '타이타닉', 'emojis': '🚢💔🌊'}, {'movie': '매트릭스', 'emojis': '💊🕶️💻'}, {'movie': '스타워즈', 'emojis': '🌌🚀🧔'}, {'movie': '어벤져스', 'emojis': '🦸\u200d♂️🦸\u200d♀️🌍'}, {'movie': '인셉션', 'emojis': '🌀🛌🕰️'}, {'movie': '쥬라기 공원', 'emojis': '🦖🌴🚁'}, {'movie': '라라랜드', 'emojis': '🎹💃🌆'}] input_variables=['movie'] example_prompt=ChatPromptTemplate(input_variables=['emojis', 'movie'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a machine that takes the name of the movie and makes three emojis that can represent the movie')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['movie'], template='{movie}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['emojis'], template='{emojis}'))])


In [52]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | fewshot_template | chat

def invoke_chain(movie):
    result = chain.invoke({
        "movie": movie,
    })
    memory.save_context({"input":movie}, {"output":result.content})
    print(result)

In [53]:
invoke_chain("신과함께")

content='🎶🌟🌆'


In [54]:
invoke_chain("비긴어게인")

content='🎶🎭🌟'


In [55]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human mentions "신과함께."'),
  AIMessage(content='🎶🌟🌆'),
  HumanMessage(content='비긴어게인'),
  AIMessage(content='🎶🎭🌟')]}